In [1]:
import wandb
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set_theme()
sns.set_context('paper')

%matplotlib inline

In [2]:
import os
dump_dir = os.path.abspath('../.log')

def parse_dump(dump):
#     ts = dump.get('ts')
#     z0_orig = dump.get('z0_orig')
    true_zt = dump.get('true_zt')
#     true_zt_chaos = dump.get('true_zt_chaos')
    pred_zt = dump.get('pred_zt')

    return true_zt, pred_zt

def download_runs_sweep(run_ids):
    api = wandb.Api()
    for run_id in run_ids:
        download_root = os.path.join(dump_dir, run_id)
        run = api.run("ngruver/physics-uncertainty-exps/{}".format(run_id))
        
        for f in run.files():
            if f.name != 'data.pt' and f.name != 'model.pt':
                continue
        
            fpath = os.path.join(download_root, f.name)
            if not os.path.isfile(fpath):
                f.download(root=download_root)
                
            if f.name == 'data.pt':
                data = torch.load(fpath)
            else:
                model = torch.load(fpath, map_location=torch.device('cpu'))
            
        yield run.name, run.config, model, parse_dump(data)

In [4]:
import os
import sys
from pprint import pprint

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.train.ensemble_trainer import DeepEnsembleTrainer
    
de_run_id = ["1axj7lqk"]

for _, cfg, model_dict, (true_zt, pred_zt) in download_runs_sweep(de_run_id):
    pass

os.environ['DATADIR'] = "/Users/nategruver/Desktop/"
cfg.pop('uq_type')
de_model = DeepEnsembleTrainer(**cfg).model
de_model.load_state_dict(model_dict)

tensor(1.2447e-05, dtype=torch.float64)
HNN currently assumes potential energy depends only on q
HNN currently assumes time independent Hamiltonian
tensor(1.2447e-05, dtype=torch.float64)
HNN currently assumes potential energy depends only on q
HNN currently assumes time independent Hamiltonian
tensor(1.2447e-05, dtype=torch.float64)
HNN currently assumes potential energy depends only on q
HNN currently assumes time independent Hamiltonian
tensor(1.2447e-05, dtype=torch.float64)
HNN currently assumes potential energy depends only on q
HNN currently assumes time independent Hamiltonian
tensor(1.2447e-05, dtype=torch.float64)
HNN currently assumes potential energy depends only on q
HNN currently assumes time independent Hamiltonian
tensor(1.2447e-05, dtype=torch.float64)
HNN currently assumes potential energy depends only on q
HNN currently assumes time independent Hamiltonian
tensor(1.2447e-05, dtype=torch.float64)
HNN currently assumes potential energy depends only on q
HNN currently a

RuntimeError: Error(s) in loading state_dict for DeepEnsembleModel:
	Missing key(s) in state_dict: "model.potential_net.3.0.weight", "model.potential_net.3.0.bias", "model.potential_net.4.weight", "model.potential_net.4.bias", "model.mass_net.1.0.weight", "model.mass_net.1.0.bias", "model.mass_net.2.0.weight", "model.mass_net.2.0.bias", "model.mass_net.3.0.weight", "model.mass_net.3.0.bias", "model.mass_net.4.weight", "model.mass_net.4.bias". 
	Unexpected key(s) in state_dict: "ensemble.0.d_moments.0", "ensemble.0.swag_d_moments.n_models", "ensemble.0.swag_d_moments.base.0", "ensemble.0.swag_d_moments.base.0_mean", "ensemble.0.swag_d_moments.base.0_sq_mean", "ensemble.0.potential_net.0.0.weight", "ensemble.0.potential_net.0.0.bias", "ensemble.0.potential_net.0.0.weight_mean", "ensemble.0.potential_net.0.0.weight_sq_mean", "ensemble.0.potential_net.0.0.bias_mean", "ensemble.0.potential_net.0.0.bias_sq_mean", "ensemble.0.potential_net.1.0.weight", "ensemble.0.potential_net.1.0.bias", "en